# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Making-and-Viewing-data" data-toc-modified-id="Making-and-Viewing-data-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Making and Viewing data</a></div><div class="lev2 toc-item"><a href="#Building-and-Training" data-toc-modified-id="Building-and-Training-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Building and Training</a></div><div class="lev2 toc-item"><a href="#Learn-to-predict-a-next-char-with-30-characters" data-toc-modified-id="Learn-to-predict-a-next-char-with-30-characters-03"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Learn to predict a next char with<dfn class="dictionary-of-numbers"> 30 characters</dfn></a></div><div class="lev2 toc-item"><a href="#Learn-to-use-30-chars-(1-to-30)-to-predict-30-chars-(2-to-31):-QUESTION" data-toc-modified-id="Learn-to-use-30-chars-(1-to-30)-to-predict-30-chars-(2-to-31):-QUESTION-04"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Learn to use<dfn class="dictionary-of-numbers"> 30 chars </dfn><dfn class="dictionary-of-numbers">(1 to </dfn>30) to predict<dfn class="dictionary-of-numbers"> 30 chars </dfn><dfn class="dictionary-of-numbers">(2 to </dfn>31): QUESTION</a></div><div class="lev2 toc-item"><a href="#Evaluating-and-Viewing-result" data-toc-modified-id="Evaluating-and-Viewing-result-05"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Evaluating and Viewing result</a></div><div class="lev2 toc-item"><a href="#Testing" data-toc-modified-id="Testing-06"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>Testing</a></div>

## Making and Viewing data

In [13]:
# create dataset, to change seq_len, go inside make_data.py to change directly
!python utils_func/make_data.py

train :
sample Index: 0 13300
dims:
(13300, 30, 30)
(13300, 30)
validate :
sample Index: 13300 14131
dims:
(831, 30, 30)
(831, 30)
test :
sample Index: 14131 14962
dims:
(831, 30, 30)
(831, 30)
evaluate :
sample Index: 14962 15793
dims:
(831, 30, 30)
(831, 30)


In [20]:
# Let's see what the preprocessed data sample look like before entering model
!python utils_func/view_data.py



peek at train:
------
"the project gutenberg ebook of" --> " "
{'in_seq': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Building and Training

In [41]:
%%writefile char_rnn_basic_defaults.yaml


# ANSWER: 
# solution1: error display when change from lstm to gru works 
# solution2: change backend to tensorflow backend, can help with the model loss is NaN issue too

---

settings:
    
                                    #backend(http://kur.deepgram.com/specification.html?highlight=backend#setting-the-backend)
  backend: 
    name: pytorch

         
                
  vocab:                                         
    size: 30                          # This cannot be changed, it is fixed with dataset  




  rnn:
    size: 128                                    # num_neurons of a rnn/lstm layer
    depth: 3                                     # num_rnn_layers for this RNN model
        


model:
  - input: in_seq


  - for:
      range: "{{ rnn.depth - 1 }}"
      iterate:
        - recurrent:
            size: "{{ rnn.size }}"
            type: lstm
            sequence: yes                   # yes, return 30 characters on a sequence
            bidirectional: no
        - batch_normalization
#         - dropout: "{{drop_neurons}}"

  - recurrent:
      size: "{{ rnn.size }}"
      type: lstm
      sequence: "{{last_seq}}"                          # no, just return the last character on a sequence
      bidirectional: no
#   - batch_normalization
#   - dropout: "{{drop_neurons}}"
    

  - dense: "{{dense.size}}"                    # now it is like 30 class-classification problem, 
                                                 # that's why we need 30 neurons here

  - activation: softmax

  - output: out_char                              # make a name of output layer
           

loss:
  - target: out_char
    name: categorical_crossentropy

train:
  data:
    - jsonl: data/train.jsonl
        
  epochs: "{{ num_epochs|default(5) }}"     
    
  weights:
    initial: t3_dlnd/best.w.kur                  # QUESTION: can I create a variable to replace `t3_dlnd` whenever I tweak model?
    best: t3_dlnd/best.w.kur
    last: t3_dlnd/last.w.kur
        
  log: t3_dlnd/log
    
  optimizer: 
    name: adam    
#     clip:                                              
#       norm: "{{grad_clip}}"   
        
  hooks:                                   
    - plot: t3_dlnd/loss.png
        
                               

validate:
    
  data:
    - jsonl: data/validate.jsonl
        
  weights: t3_dlnd/best.w.kur


test:
    
  data:
    - jsonl: data/test.jsonl
        
  weights: t3_dlnd/best.w.kur


evaluate:
    
  data:
    - jsonl: data/evaluate.jsonl
        
  weights: t3_dlnd/best.w.kur

  destination: t3_dlnd/output.pkl

Overwriting char_rnn_basic_defaults.yaml


## Learn to predict a next char with 30 characters

In [44]:
%%writefile char_rnn_basic_fluid.yaml

---
settings: 
  last_seq: no  
    
  dense:
    size: 30
    
  num_epochs: 1                    # leave it empty means inf number of epochs
                                 # so to use default value, just comment this line out
#   drop_neurons: 0.5
#   grad_clip: 5

include: char_rnn_basic_defaults.yaml
...

Overwriting char_rnn_basic_fluid.yaml


In [45]:
!kur -v train char_rnn_basic_fluid.yaml

[INFO 2017-03-08 17:10:45,684 kur.kurfile:699] Parsing source: char_rnn_basic_fluid.yaml, included by top-level.
[INFO 2017-03-08 17:10:45,689 kur.kurfile:699] Parsing source: char_rnn_basic_defaults.yaml, included by char_rnn_basic_fluid.yaml.
[INFO 2017-03-08 17:10:45,703 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-08 17:10:45,726 kur.loggers.binary_logger:71] Loading log data: t3_dlnd/log
[INFO 2017-03-08 17:10:49,569 kur.backend.backend:80] Creating backend: pytorch
[INFO 2017-03-08 17:10:49,569 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-08 17:10:49,633 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-08 17:10:49,633 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-08 17:10:49,633 kur.model.model:281] Connecting the model graph.
[INFO 2017-03-08 17:10:49,640 kur.model.model:285] Model inputs:  in_seq
[INFO 2017-03-08 17:10:49,641 kur.model.model:286] Model outputs: out_char
[INFO 2017-03-08 17:10:49,647 kur.model.e

**QUESTION: How to generate long text** like karpathy did in [his post](https://hyp.is/KYLatAPxEee89XOhgBMapQ/karpathy.github.io/2015/05/21/rnn-effectiveness/)?
- What I did above is similar to graph 1 on the left (SEE figure below), I use a model use 30 characters (index 1 to index 30) to predict the next (31th) character (in the graph I used 50, just so you know)
- What I want to achieve in the next kurfile below is graph 2 in the middle, which is a model to use 30 characters (say index 1 to 30th) to predict 30 characters with index (2 to 31th)
- to generate a long text like what karpathy did in the post link above, should I train my model like graph2 with much longer seq_len, say 200? Is it how karpathy got the long text??



In [1]:
from IPython.display import Image
Image(url='https://lh3.googleusercontent.com/Ybix_t5PBJaST4icd_BY8WvvFRfieba58Wkup2S1gpRf3mFdxp00iOtpWKdh6ptM9uexoTORCr8mfQL4Qy4oeiIzIZxtGFXgZsSA34g3teyBhJ_fxqQphwxpPyCsa7q4t530EE4BT8XiceQwoAuCl1PhBFR-2UA4NfZZiL5hyrVeXOdoRSzZ1jl45MekOaHad-9j1A5_PNMeOZz8BmLx3Uc5xFWNrdlZ0rSql385Sum81rdntmMUWY8MnN7LYzr96IlgN3Hxd0uQZkbc5pn6YD8LTew_7xlSGBiYxqIoc2qfjmFd_mWr-RP445kq_firczCmjHows4aOsKw0yK-DaOBCGc-ovUv9c_1OFqYQYKGHk6VTMf3TF3gP4X2g0tOk4K6zHP3MSTZGpvEj1-HGCq8kVPwBXVnj1jI-nKnLwIiVePlbYt8qAls4uhoOXGJb5Nm4gotIoOTsBdR1NRYiULuUghcIjbWy22uCA-2CbO59oOLf-dXXHO9brhOutFUP5Mazt3BQk6SOUQbf7tqbqsRWlp7iMXdC0kwXEeUiNGSw0caNKOPjhg2z-GBAqDoCrmuc9YZQ1UyI-UZr-ppyNLVXaecRfjePylfNyp1eFjx8p2ccJ5xo=w1912-h1352-no')

## Learn to use 30 chars (1 to 30) to predict 30 chars (2 to 31): QUESTION
**What I changed to the model above**
- I cleaned the directory of t3_dlnd first
- set `last_seq` to yes
- set `dense` to [30, 30]

**Error**
- see below

In [46]:
%%writefile char_rnn_basic_fluid.yaml

---
settings: 
  last_seq: yes  
    
  dense:
    size: [30,30]
    
  num_epochs: 1                    # leave it empty means inf number of epochs
                                 # so to use default value, just comment this line out
#   drop_neurons: 0.5
#   grad_clip: 5

include: char_rnn_basic_defaults.yaml
...

Overwriting char_rnn_basic_fluid.yaml


In [47]:
!kur -v train char_rnn_basic_fluid.yaml

[INFO 2017-03-08 17:16:51,521 kur.kurfile:699] Parsing source: char_rnn_basic_fluid.yaml, included by top-level.
[INFO 2017-03-08 17:16:51,525 kur.kurfile:699] Parsing source: char_rnn_basic_defaults.yaml, included by char_rnn_basic_fluid.yaml.
[INFO 2017-03-08 17:16:51,538 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-08 17:16:51,567 kur.loggers.binary_logger:107] Log does not exist. Creating path: t3_dlnd/log
[INFO 2017-03-08 17:16:55,064 kur.backend.backend:80] Creating backend: pytorch
[INFO 2017-03-08 17:16:55,064 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-08 17:16:55,141 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-08 17:16:55,142 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-08 17:16:55,142 kur.model.model:281] Connecting the model graph.
[INFO 2017-03-08 17:16:55,147 kur.model.model:285] Model inputs:  in_seq
[INFO 2017-03-08 17:16:55,147 kur.model.model:286] Model outputs: out_char
[INFO 2017-03-08 17:16:

## Evaluating and Viewing result

In [21]:
!kur -v evaluate char_rnn_basic_fluid.yaml

[INFO 2017-03-08 16:39:58,603 kur.kurfile:699] Parsing source: char_rnn_basic_fluid.yaml, included by top-level.
[INFO 2017-03-08 16:39:58,607 kur.kurfile:699] Parsing source: char_rnn_basic_defaults.yaml, included by char_rnn_basic_fluid.yaml.
[INFO 2017-03-08 16:39:58,621 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-08 16:39:58,866 kur.backend.backend:80] Creating backend: pytorch
[INFO 2017-03-08 16:39:58,867 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-08 16:39:58,956 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-08 16:39:58,956 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-08 16:39:58,956 kur.model.model:281] Connecting the model graph.
[INFO 2017-03-08 16:39:58,962 kur.model.model:285] Model inputs:  in_seq
[INFO 2017-03-08 16:39:58,962 kur.model.model:286] Model outputs: out_char
Evaluating: 100%|███████████████████████| 831/831 [00:01<00:00, 507.33samples/s]
[INFO 2017-03-08 16:40:00,608 kur.model.hooks.outp

In [24]:
# viewing result from evaluation: 
!python utils_func/view_outputs.py 't3_dlnd/output.pkl'

"ng your time with me. mr. bing" --> "l"
                                   CORRECT
"g your time with me. mr. bingl" --> "e"
                                   CORRECT
" your time with me. mr. bingle" --> "y"
                                   CORRECT
"your time with me. mr. bingley" --> " "
                                   CORRECT
"our time with me. mr. bingley " --> "w"
                                   INCORRECT (f)
"ur time with me. mr. bingley f" --> "o"
                                   CORRECT
"r time with me. mr. bingley fo" --> "r"
                                   INCORRECT (l)
" time with me. mr. bingley fol" --> "l"
                                   CORRECT
"time with me. mr. bingley foll" --> "y"
                                   INCORRECT (o)
"ime with me. mr. bingley follo" --> "w"
                                   CORRECT
accuracy = 0.5006016847172082


## Testing 

In [25]:
!kur -v test char_rnn_basic_fluid.yaml

[INFO 2017-03-08 16:42:02,470 kur.kurfile:699] Parsing source: char_rnn_basic_fluid.yaml, included by top-level.
[INFO 2017-03-08 16:42:02,474 kur.kurfile:699] Parsing source: char_rnn_basic_defaults.yaml, included by char_rnn_basic_fluid.yaml.
[INFO 2017-03-08 16:42:02,489 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-08 16:42:02,738 kur.backend.backend:80] Creating backend: pytorch
[INFO 2017-03-08 16:42:02,739 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-08 16:42:02,832 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-08 16:42:02,832 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-08 16:42:02,833 kur.model.model:281] Connecting the model graph.
[INFO 2017-03-08 16:42:02,839 kur.model.model:285] Model inputs:  in_seq
[INFO 2017-03-08 16:42:02,839 kur.model.model:286] Model outputs: out_char
Testing, loss=1.633: 100%|██████████████| 831/831 [00:01<00:00, 516.77samples/s]
[INFO 2017-03-08 16:42:04,453 kur.model.executor:1